# Test deployed web application

In [24]:
import base64
import json
import requests

In [13]:
app_url = '13.90.129.123:88'

In [14]:
scoring_url = 'http://{}/score'.format(app_url)
version_url = 'http://{}/version'.format(app_url)

In [15]:
version_url

'http://13.90.129.123:88/version'

In [16]:
!curl $version_url # Reports the tensorflow version

1.4.1

In [22]:
health_url = 'http://{}/'.format(app_url)
!curl $health_url

Healthy

In [23]:
def img_to_json(img_path):
    with open(img_path, 'rb') as file:
        encoded = base64.b64encode(file.read())
    img_dict = {img_path: encoded.decode('utf-8')}
    body = json.dumps(img_dict)
    return body

In [25]:
img_path = 'elephant.jpg'
body = img_to_json(img_path)

In [26]:
jsonimg = json.dumps({'input':'{}'.format(body)})
jsonimg[:100] 

'{"input": "{\\"elephant.jpg\\": \\"/9j/4SHNRXhpZgAATU0AKgAAAAgABwESAAMAAAABAAEAAAEaAAUAAAABAAAAYgEbAAUA'

In [28]:
headers = {'content-type': 'application/json'}
r = requests.post(scoring_url, data=jsonimg, headers=headers)
print(r)
r.json()

<Response [200]>


{'result': "([{'elephant.jpg': [('n02504458', 'African_elephant', 0.51476455), ('n01871265', 'tusker', 0.4063379), ('n02504013', 'Indian_elephant', 0.077737026)]}], 'Predictions took 2112.76 ms')"}